In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :hPC;
const FLTYPE = Float32;
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [ ]:
IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");

In [ ]:
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

## Old workflow
Every array is using `Point3f0`

```julia
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of Point3f0
reference_array = vertices(reference_pc);
reading_array = vertices(reading_pc);
#KD tree for the reference array
#kdtree_ref = KDTree(reference_array);

#Sampling random elements from the reference array
sampled_points_ref = randomSamplePoints(reference_array,0.5);
#Building KD tree for the sampled points
kdtree_sampled_ref = KDTree(sampled_points_ref);

#pairing the points in the two meshes
indexer,fl_traits = createKnnPairArray(reading_array,kdtree_sampled_ref,sampled_points_ref,true);
#rejecting the worst 20% based on the distance
rej_is,rej_fltrs = rejectWorstPercent(indexer,fl_traits,0.2);
```

## New workflow
Using Homogeneous vector instead of `Point3f0`

Ötlet: amíg lehet csak az az indexeket használni és csak ahol muszáj, ott számolni a konkrét mátrixokkal
### Workflow lépései:
1. [x] reading és ref mesh-t homogén koordinátás SVector/MVector**(?)** tömbbé alakítani
1. [x] a teljes referencia tömbből kd-tree
### Iteráció
1. [x] reading tömbből sampling
1. [x] samplinged tömbhöz párkeresés a fából
1. ##### Worst `x`% rejection
  * [x] párok sorbarendezése távolság szerint
  * [x] a sorbarendezett párok `x`%-nak kidobása
1. ##### 2.5sigma rejection
  * [ ] szórás számítása
  * [ ] 2.5 sigmánál nagyobbak kidobása
1. [x] A\b összeállítása és megoldása
1. [x] transzformáció a megoldással
1. [x] transzformáció mentése vizualizációhoz
1. [ ] kilépési feltétel check

### Iteráció vége

11. [ ] place in MeshCat
12. [ ] animate in MeshCat


In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of SVector
reference_array = convert2HomogeneousArray(vertices(reference_pc),FLTYPE);
reading_array = convert2HomogeneousArray(vertices(reading_pc),FLTYPE);
#Sampling random elements from the reference and the reading array
sampled_points_ref = randomSamplePoints(reference_array,50);
sampled_points_red = randomSamplePoints(reading_array,50);
#Building KD tree for the sampled ref points and the whole ref array
kdtree_sampled_ref = KDTree(sampled_points_ref);
kdtree_ref = KDTree(reference_array);

#Iterate from here

#pairing the points in the two meshes
indM, traitM = createKnnPairArray(sampled_points_red,kdtree_ref);
#sorting the indexes by the distances in the trait matrix
s_indM, s_it = sortIndexes(indM, traitM[:,2]);
#remove the worst 10%
ch_ind, ch_it = chopEndOfArray(20,s_indM,s_it);

### Newnew workflow with matrix instead of array of array

In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);

In [ ]:
function firstICP(ref_mesh, read_mesh, iternum; fltype = Float32, sampleN = 50, chopN = 20, v = true)
    #Matrixes
    ref_mat = convert2HomCoordMatrix(vertices(reference_pc), fltype)
    red_mat = convert2HomCoordMatrix(vertices(reading_pc), fltype)
    #Transposed reference matrix
    ref_mtp = giveTranspose(ref_mat)
    red_mtp = giveTranspose(red_mat)

    #Sampling random indexes for the reference and reading arrays
    #si_ref = randomSampleIndexes(50, ref_mtp, 2)
    #si_red = randomSampleIndexes(50, red_mtp, 2)

    #Building KD tree for the whole ref array (transposed matrix must be used)
    kdtree_ref = KDTree(ref_mtp);
    
    firstDict, ujar = basicIteration(red_mtp, kdtree_ref, ref_mat, iternum, sampleN=sampleN, chopN=chopN, v=v);
    return firstDict, red_mtp, ujar
end

In [ ]:
function basicIteration(read_vert, kdtree_r, ref_mhor, itnum; sampleN = 50, chopN = 20, v = true)
    trDict = Dict{Int,Any}()
    for i in 1:itnum
        #selecting random % of the reading
        si_red = randomSampleIndexes(sampleN, read_vert, 2);
        #pairing the points in the two meshes
        indM, traitM = createKnnPairArray(read_vert[:,si_red], kdtree_r);
        #sorting the indexes by the distances in the trait matrix
        s_indM, s_it = sortIndexes(indM, traitM[:,2]);
        #remove the worst %
        ch_ind, ch_it = chopEndOfArray(chopN, s_indM,s_it);
        #make the transformation matrix
        coordTR = transpose(read_vert[:,ch_ind[:,2]])\ref_mhor[ch_ind[:,1],:];
        #transzforálmi az mtp-t kell
        coordTR[:,4] = [0,0,0,1]
        read_vert = transpose(coordTR) * read_vert;
        trDict[i] = transpose(coordTR)
        if v && i%100 == 0
            print("\r $i'th iteration")
        end
    end
    return trDict, read_vert
end

In [ ]:
dicti, tred_mesh, trmeshuj = firstICP(reference_pc, reading_pc, 1000);

In [ ]:
newmesh = makeMeshfromMatrix(transpose(tred_mesh));

In [ ]:
newmesh_tr = makeMeshfromMatrix(transpose(trmeshuj));

In [ ]:
newmesh_tr

In [ ]:
placeMeshInLife(vis, newmesh, "tred")

In [ ]:
placeMeshInLife(vis, newmesh_tr, "trededd", "orange")

In [ ]:
@benchmark firstICP(reference_pc, reading_pc, 1000, v = false)

In [ ]:
function affinMapList(dicti, accumulate = false)
    aff_arr = Array{AffineMap}(undef,length(dicti))
    for i in 1:length(dicti)
        aff_arr[i] = AffineMap(SMatrix{3,3}(dicti[i][1:3,1:3]), SVector{3}(dicti[i][1:3,4]))
    end
    return aff_arr
end

In [ ]:
affinMapList(dicti)

## TODOs:
* mtp tömb allokálódik a belső ciklusban?
* affin tr. konverzió ellenőrzése
* affin tr. akkumulálása
* vizualizáció
* animáció